In [ ]:
import polars as pl
import polars.selectors as cs

In [ ]:
df = pl.read_csv("../data/TestLookUp/input/report (16).csv").drop("legacy_user_id")
df_cols = {
    "id",
    "activity_flow_submission_id",
    "activity_scheduled_time",
    "activity_start_time",
    "activity_end_time",
    "flag",
    "secret_user_id",
    "userId",
    "source_user_subject_id",
    "source_user_secret_id",
    "source_user_nickname",
    "source_user_relation",
    "source_user_tag",
    "target_user_subject_id",
    "target_user_secret_id",
    "target_user_nickname",
    "target_user_tag",
    "input_user_subject_id",
    "input_user_secret_id",
    "input_user_nickname",
    "activity_id",
    "activity_name",
    "activity_flow_id",
    "activity_flow_name",
    "item",
    "item_id",
    "response",
    "prompt",
    "options",
    "version",
    "rawScore",
    "reviewing_id",
    "event_id",
    "timezone_offset",
}
id_cols = {
    "id",
    "activity_flow_submission_id",
    "activity_scheduled_time",
    "activity_start_time",
    "activity_end_time",
    "flag",
    "secret_user_id",
    "userId",
    "source_user_subject_id",
    "source_user_secret_id",
    "source_user_nickname",
    "source_user_relation",
    "source_user_tag",
    "target_user_subject_id",
    "target_user_secret_id",
    "target_user_nickname",
    "target_user_tag",
    "input_user_subject_id",
    "input_user_secret_id",
    "input_user_nickname",
    "activity_id",
    "activity_name",
    "activity_flow_id",
    "activity_flow_name",
    "version",
    "reviewing_id",
    "event_id",
    "timezone_offset",
}
response_cols = df_cols - id_cols
print(response_cols)
ssdf = (
    df.select(pl.all().exclude(response_cols))
    .rename(
        {
            "Final SubScale Score": "subscale__final_score",
            "Optional text for Final SubScale Score": "subscale_text__final_score",
        }
    )
    .select(
        ~cs.matches("^Optional text for .*$"),
        pl.col("^Optional text for .*$").name.map(
            lambda n: f"subscale_text__{n[18:].replace(' ', '_')}"
        ),
    )
    .select(
        cs.by_name(id_cols),
        cs.starts_with("subscale_"),
        (~(cs.by_name(id_cols) | cs.starts_with("subscale_"))).name.map(
            lambda ss: f"subscale__{ss.replace(" ", "_")}"
        ),
    )
    .unpivot(index=id_cols, variable_name="item", value_name="response")
    .filter(pl.col("response").is_not_null())
    .with_columns(item_id=None, prompt=None, options=None, rawScore=None)
)

# df = pl.concat([df.select(df_cols), ssdf])

with pl.Config(tbl_rows=-1):
    print(df_cols - set(ssdf.columns))
    print(ssdf)